In [39]:
#data science and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#creation of dataset
import lib.ml_workflow.create_dataset as cds
from lib.raman_lib.misc import load_data

#quality control
import lib.ml_workflow.quality_control as qc
from lib.raman_lib.preprocessing import RangeLimiter
from lib.raman_lib.visualization import plot_spectra_peaks
from lib.raman_lib.spectra_scoring import score_names

#preprocessing
from lib.ml_workflow.preprocess_data import preprocess

#model creation
from sklearn.model_selection import StratifiedKFold, cross_validate, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import FeatureAgglomeration
from sklearn.decomposition import PCA, NMF
from sklearn.pipeline import Pipeline
from lib.raman_lib.preprocessing import PeakPicker

#file handling
from pathlib import Path
import os

In [40]:
# define the paths to all experiment data
# each dir contains all the files associated with that experiment
# each file has a prefix that indicates the group and the measurement --> group_measurement_.....
path_etoposide = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide"
path_resveratrol = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol"
path_both = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Both"
path_control = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Control"

# Define parameters
In order to function properly, the provided code depends on predefined parameters like output paths, limits and thresholds for the quality control, ...
## Define data paths
Define the location of the data, and where quality-controlled and preprocessed data should be stored. Both of them rely on a unique file-prefix that describes the data being analyzed.

In [41]:
FILE_PREFIX = "HL_428_E_R"
DATASET_OUT = "./data/" + FILE_PREFIX + ".csv"
RESULT_DIR = "./result/" + FILE_PREFIX
QC_OUT = RESULT_DIR + "/" + FILE_PREFIX + "_qc.csv"
PREP_OUT = RESULT_DIR + "/" + FILE_PREFIX + "_preprocessed.csv"


## Define quality scoring parameters
The quality control only uses peaks in a given interval, recognizes peaks via a filter (Sav-Gol) and scores them based on some metrics. Finally, the best N spectra are selected.
### Spectral Range Limits

In [42]:
QC_LIM_LOW = 450
QC_LIM_HIGH = 1650

### Peak Detection

In [43]:
QC_WINDOW = 35
QC_THRESHOLD = 0.001
QC_MIN_HEIGHT = 50

### Scoring

In [44]:
QC_SCORE = 1
QC_PEAKS = 1

### Number of spectra to keep

In [45]:
QC_NUM = 300

## Define Preprocessing Parameter
### Spectral Range Limits

In [46]:
PREP_LIM_LOW = QC_LIM_LOW
PREP_LIM_HIGH = QC_LIM_HIGH

### Window-width for smoothing

In [47]:
PREP_WINDOW = 15

## Settings for Cross Validation

In [48]:
SCORING = [ 'accuracy', 'f1']
N_TRIALS = 20
N_FOLDS = 5
N_CORES = -1

# Create the dataset
Create the dataset using the implementation provided by D. Zimmermann.
For the creation of the dataset, the two source dirs, as well as the desired labels are needed.
Furthermore, an output directory is needed, to store the created dataset 

In [49]:
datadir = Path(DATASET_OUT).parent
if not os.path.exists(datadir):
    os.makedirs(datadir)

dataset = cds.create_dataset([path_etoposide, path_resveratrol], ['Etoposide', 'Resveratrol'])
dataset.to_csv(DATASET_OUT, index=False)

root - INFO - Loading data
root - INFO - Loading data
root - INFO - Loading data
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol
root - INFO - Finished loading data.
root - INFO - Finished loading data.
root - INFO - Finished loading data.


# Do quality control
Asses the spectra based on their quality, and remove low quality spectra

In [50]:
path_in = Path(DATASET_OUT)
path_out = Path(RESULT_DIR)

if not os.path.exists(path_out):
    os.makedirs(path_out)

path_out_data = path_out / (path_in.stem + "_qc.csv")
path_out_scores = path_out / (path_in.stem + "_qc_scores.csv")

data = pd.read_csv(path_in)

data_out, _, score_dict = qc.score_sort_spectra(data,
                                                n=QC_NUM,
                                                limits=[QC_LIM_LOW, QC_LIM_HIGH],
                                                bl_method="asls",
                                                sg_window=QC_WINDOW,
                                                threshold=QC_THRESHOLD,
                                                min_height=QC_MIN_HEIGHT,
                                                score_measure=QC_SCORE,
                                                n_peaks_influence=QC_PEAKS,
                                                detailed=True)

visualize = False
if visualize:
    data_vis = data.drop(columns=["label", "file"]).values.astype(float)
    wns_vis = data.drop(columns=["label", "file"]).columns.astype(float)

    rl = RangeLimiter(lim=[QC_LIM_LOW, QC_LIM_HIGH],
                      reference=wns_vis)

    data_rl = rl.fit_transform(data_vis)
    wns_rl = wns_vis[rl.lim_[0]:rl.lim_[1]]

    plot_spectra_peaks(wns_rl,
                       data_rl,
                       score_dict["peak_pos"],
                       labels=score_dict["total_scores"])

data_out.to_csv(path_out_data, index=False)

pd.DataFrame({score_names[QC_SCORE]: score_dict["intensity_scores"],
              "N Peaks": score_dict["peak_scores"]}).to_csv(
    path_out_scores, index=False
)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Preprocess the data

In [51]:
path_in = Path(QC_OUT)
path_out = Path(RESULT_DIR)

filename = path_in.stem.removesuffix("_qc")

if not os.path.exists(path_out):
    os.makedirs(path_out)

path_out = path_out / (filename + "_preprocessed.csv")

data = load_data(QC_OUT)

data_prep = preprocess(data, limits=[PREP_LIM_LOW, PREP_LIM_HIGH], sg_window=PREP_WINDOW)



data_prep.to_csv(path_out, index=False)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Implement Leave P Out CV

In [52]:
path_in = PREP_OUT
#path_out = Path(args.out)

#filename = path_in.stem

data = load_data(path_in)

X = data.drop(columns=["label", "file"])
wns = np.asarray(X.columns.astype(float))
X = np.asarray(X)
y = np.array(data.label)
y, y_key = pd.factorize(y)

stk = StratifiedKFold(n_splits=10)

## Baseline with LDA alone

In [53]:
clf = LinearDiscriminantAnalysis()
cross_validate(clf, X, y, cv=stk, scoring=SCORING, )

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


{'fit_time': array([0.24509335, 0.32961321, 0.24941993, 0.24233007, 0.33181787,
        0.26399994, 0.25281   , 0.25865412, 0.2530067 , 0.25976014]),
 'score_time': array([0.0016377 , 0.00164986, 0.00151205, 0.00169873, 0.00152516,
        0.00154614, 0.00152493, 0.00211883, 0.00165033, 0.00151491]),
 'test_accuracy': array([0.73333333, 0.83333333, 0.71666667, 0.81666667, 0.76666667,
        0.73333333, 0.71666667, 0.73333333, 0.7       , 0.66666667]),
 'test_f1': array([0.75      , 0.82758621, 0.71186441, 0.8358209 , 0.78125   ,
        0.73333333, 0.73015873, 0.73333333, 0.71875   , 0.6875    ])}

In [54]:
clf = Pipeline([("pca", PCA()),
                ("lda", LinearDiscriminantAnalysis())])
cross_validate(clf, X, y, cv=stk, scoring=SCORING)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


{'fit_time': array([0.36269808, 0.35233283, 0.30268097, 0.31479907, 0.32101297,
        0.35075879, 0.31304789, 0.3246491 , 0.43368196, 0.43592286]),
 'score_time': array([0.00484467, 0.00441217, 0.00556493, 0.00466895, 0.00420094,
        0.00456595, 0.00472808, 0.00443196, 0.00571704, 0.00458908]),
 'test_accuracy': array([0.48333333, 0.51666667, 0.46666667, 0.55      , 0.53333333,
        0.48333333, 0.51666667, 0.51666667, 0.41666667, 0.5       ]),
 'test_f1': array([0.47457627, 0.52459016, 0.46666667, 0.52631579, 0.5625    ,
        0.47457627, 0.52459016, 0.49122807, 0.40677966, 0.51612903])}

In [55]:
"""clf = Pipeline([("nmf", NMF(init="nndsvda", tol=1e-2, max_iter=5000)),
                ("lda", LinearDiscriminantAnalysis())])
cross_validate(clf, X, y, cv=stk, scoring=SCORING)"""

'clf = Pipeline([("nmf", NMF(init="nndsvda", tol=1e-2, max_iter=5000)),\n                ("lda", LinearDiscriminantAnalysis())])\ncross_validate(clf, X, y, cv=stk, scoring=SCORING)'

In [56]:
clf = Pipeline([("agglo", FeatureAgglomeration(connectivity=np.diag(np.ones(len(wns))) +
                                                            np.diag(np.ones(len(wns)-1), 1) +
                                                            np.diag(np.ones(len(wns)-1), -1))),
                ("lda", LinearDiscriminantAnalysis())])
cross_validate(clf, X, y, cv=stk, scoring=SCORING)


{'fit_time': array([0.19374609, 0.14283013, 0.139575  , 0.140064  , 0.13956809,
        0.13995314, 0.14832902, 0.14758205, 0.14164996, 0.14063096]),
 'score_time': array([0.00241876, 0.00238204, 0.00237179, 0.00232124, 0.00231409,
        0.00264621, 0.00236607, 0.00234389, 0.00231814, 0.00228   ]),
 'test_accuracy': array([0.55      , 0.63333333, 0.5       , 0.71666667, 0.78333333,
        0.63333333, 0.53333333, 0.68333333, 0.7       , 0.63333333]),
 'test_f1': array([0.54237288, 0.64516129, 0.48275862, 0.73846154, 0.80597015,
        0.62068966, 0.53333333, 0.6779661 , 0.64      , 0.5       ])}

In [57]:
clf = Pipeline([("peaks", PeakPicker()),
                ("lda", LinearDiscriminantAnalysis())])
cross_validate(clf, X, y, cv=stk, scoring=SCORING)


{'fit_time': array([0.00474906, 0.0046339 , 0.00400305, 0.00392509, 0.00433326,
        0.0042851 , 0.00555587, 0.00409031, 0.00399613, 0.00390506]),
 'score_time': array([0.00138474, 0.00135326, 0.00130486, 0.00129104, 0.00128579,
        0.00145197, 0.00138402, 0.00124788, 0.00133872, 0.0012579 ]),
 'test_accuracy': array([0.51666667, 0.73333333, 0.78333333, 0.73333333, 0.75      ,
        0.73333333, 0.75      , 0.78333333, 0.73333333, 0.63333333]),
 'test_f1': array([0.61333333, 0.73333333, 0.80597015, 0.73333333, 0.76923077,
        0.75      , 0.75409836, 0.78688525, 0.71428571, 0.52173913])}